### Imports

In [1]:
import sys
import pandas as pd
import numpy as np
import json
import re

### Pfade & Encoding

In [2]:
data_path = "./Hauptordner"
encoding = "utf-8"

### Zufallsseed für Reproduzierbarkeit

In [3]:
RANDOM_SEED = 42


### Dataset 4: summoner_spell_info – Grundchecks
- **Form & Größe**
- **Spaltennamen** und **Datentypen**
- **Erste Zeilen**
- **Zufällige Stichprobe**
- **Statistische Übersicht**
- **Fehlende Werte**
- **Duplikate**
- **Eindeutigkeit von Schlüsseln**
- **Min-/Max-Werte**
- **Wertebereiche**
- **Kardinalität**
- **Kategorische Verteilungen**
- **Auffälligkeiten markieren**
- **Konsistenz zwischen Spalten**
- **Komplett leere oder fast leere Spalten**
- **Datums-/Zeitspalten prüfen**
- **Zeichenketten in kategorischen Spalten prüfen**
- **Version prüfen**
- **Summoner-Level-Bereich prüfen**
- **Beschreibung auf Vollständigkeit prüfen**
- **Sonderzeichen in Namen prüfen**
- **Key-Format prüfen**
- **Doppelte Beschreibungen prüfen**
- **Ausreißer Checks**

### Form & Größe

In [4]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\summoner_spell_info.json"
df_spells = pd.read_json(pfad)
df_spells.shape

(17, 3)

### Spaltennamen und Datentypen

In [5]:
df_spells.dtypes

type       object
version    object
data       object
dtype: object

### Erste Zeilen

In [6]:
df_spells.head()

,type,version,data
1,summoner,7.17.2,"{'id': 1, 'summonerLevel': 6, 'name': 'Cleanse..."
3,summoner,7.17.2,"{'id': 3, 'summonerLevel': 4, 'name': 'Exhaust..."
4,summoner,7.17.2,"{'id': 4, 'summonerLevel': 8, 'name': 'Flash',..."
6,summoner,7.17.2,"{'id': 6, 'summonerLevel': 1, 'name': 'Ghost',..."
7,summoner,7.17.2,"{'id': 7, 'summonerLevel': 1, 'name': 'Heal', ..."


### Zufällige Stichprobe

In [7]:
df_spells.sample(5, random_state=1)

,type,version,data
6,summoner,7.17.2,"{'id': 6, 'summonerLevel': 1, 'name': 'Ghost',..."
33,summoner,7.17.2,"{'id': 33, 'summonerLevel': 1, 'name': 'Nexus ..."
13,summoner,7.17.2,"{'id': 13, 'summonerLevel': 1, 'name': 'Clarit..."
4,summoner,7.17.2,"{'id': 4, 'summonerLevel': 8, 'name': 'Flash',..."
12,summoner,7.17.2,"{'id': 12, 'summonerLevel': 6, 'name': 'Telepo..."


### Statistische Übersicht

In [8]:
df_spells.describe

<bound method NDFrame.describe of         type version                                               data
1   summoner  7.17.2  {'id': 1, 'summonerLevel': 6, 'name': 'Cleanse...
3   summoner  7.17.2  {'id': 3, 'summonerLevel': 4, 'name': 'Exhaust...
4   summoner  7.17.2  {'id': 4, 'summonerLevel': 8, 'name': 'Flash',...
6   summoner  7.17.2  {'id': 6, 'summonerLevel': 1, 'name': 'Ghost',...
7   summoner  7.17.2  {'id': 7, 'summonerLevel': 1, 'name': 'Heal', ...
11  summoner  7.17.2  {'id': 11, 'summonerLevel': 10, 'name': 'Smite...
12  summoner  7.17.2  {'id': 12, 'summonerLevel': 6, 'name': 'Telepo...
13  summoner  7.17.2  {'id': 13, 'summonerLevel': 1, 'name': 'Clarit...
14  summoner  7.17.2  {'id': 14, 'summonerLevel': 10, 'name': 'Ignit...
21  summoner  7.17.2  {'id': 21, 'summonerLevel': 4, 'name': 'Barrie...
30  summoner  7.17.2  {'id': 30, 'summonerLevel': 1, 'name': 'To the...
31  summoner  7.17.2  {'id': 31, 'summonerLevel': 1, 'name': 'Poro T...
32  summoner  7.17.2  {'id': 3

### Fehlende Werte

In [9]:
df_spells.isnull().sum()

type       0
version    0
data       0
dtype: int64

### Duplikate

In [10]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\summoner_spell_info.json"
data = pd.read_json(pfad)
spell_dict = data.loc[data.index[0], 'data']
df_spells = pd.json_normalize(spell_dict.values())

df_spells.duplicated().sum()

np.int64(0)

### Eindeutigkeit von Schlüsseln

In [11]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\summoner_spell_info.json"
data = pd.read_json(pfad)
spell_dict = data['data']
df_spells = pd.json_normalize(spell_dict.values)

df_spells['key'].is_unique

True

### Min-/Max-Werte

In [12]:
df_spells.describe().loc[['min', 'max']]

,id,summonerLevel
min,1.0,1.0
max,36.0,10.0


### Wertebereiche

In [13]:
df_spells.describe()

,id,summonerLevel
count,17.000000,17.000000
mean,19.000000,3.411765
std,12.980755,3.355198
min,1.000000,1.000000
25%,7.000000,1.000000
50%,14.000000,1.000000
75%,32.000000,6.000000
max,36.000000,10.000000


### Kardinalität

In [14]:
df_spells.nunique()

id               17
summonerLevel     5
name             15
key              17
description      15
dtype: int64

### Kategorische Verteilungen

In [15]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\summoner_spell_info.json"
with open(pfad, encoding='utf-8') as f:
    raw = json.load(f)
spell_dict = raw['data']
spell_liste = list(spell_dict.values())
df_spells = pd.json_normalize(spell_liste)

print(df_spells.columns)
df_spells['key'].value_counts()

Index(['id', 'summonerLevel', 'name', 'key', 'description'], dtype='object')


key
SummonerBoost                   1
SummonerExhaust                 1
SummonerFlash                   1
SummonerHaste                   1
SummonerHeal                    1
SummonerSmite                   1
SummonerTeleport                1
SummonerMana                    1
SummonerDot                     1
SummonerBarrier                 1
SummonerPoroRecall              1
SummonerPoroThrow               1
SummonerSnowball                1
SummonerSiegeChampSelect1       1
SummonerSiegeChampSelect2       1
SummonerDarkStarChampSelect1    1
SummonerDarkStarChampSelect2    1
Name: count, dtype: int64

### Auffälligkeiten markieren

In [16]:
df_spells['auffällig_leer'] = df_spells.isnull().any(axis=1)
df_spells['auffällig_doppelt'] = df_spells.duplicated(subset='key', keep=False)
df_spells['auffällig_name'] = df_spells['name'].str.len() > 20
df_spells[df_spells[['auffällig_leer', 'auffällig_doppelt', 'auffällig_name']].any(axis=1)]

,id,summonerLevel,name,key,description,auffällig_leer,auffällig_doppelt,auffällig_name
13,33,1,Nexus Siege: Siege Weapon Slot,SummonerSiegeChampSelect1,"In Nexus Siege, Summoner Spells are replaced w...",False,False,True
14,34,1,Nexus Siege: Siege Weapon Slot,SummonerSiegeChampSelect2,"In Nexus Siege, Summoner Spells are replaced w...",False,False,True
15,35,1,Disabled Summoner Spells,SummonerDarkStarChampSelect1,Summoner spells are disabled in this mode.,False,False,True
16,36,1,Disabled Summoner Spells,SummonerDarkStarChampSelect2,Summoner spells are disabled in this mode.,False,False,True


### Konsistenz zwischen Spalten

In [17]:
df_spells['konsistenz_id_key'] = df_spells['id'].astype(str) == df_spells['key'].astype(str)
df_spells[~df_spells['konsistenz_id_key']]

,id,summonerLevel,name,key,description,auffällig_leer,auffällig_doppelt,auffällig_name,konsistenz_id_key
0,1,6,Cleanse,SummonerBoost,Removes all disables (excluding suppression an...,False,False,False,False
1,3,4,Exhaust,SummonerExhaust,"Exhausts target enemy champion, reducing their...",False,False,False,False
2,4,8,Flash,SummonerFlash,Teleports your champion a short distance towar...,False,False,False,False
3,6,1,Ghost,SummonerHaste,Your champion gains increased Movement Speed a...,False,False,False,False
4,7,1,Heal,SummonerHeal,Restores 90-345 Health (depending on champion ...,False,False,False,False
5,11,10,Smite,SummonerSmite,Deals 390-1000 true damage (depending on champ...,False,False,False,False
6,12,6,Teleport,SummonerTeleport,"After channeling for 4.5 seconds, teleports yo...",False,False,False,False
7,13,1,Clarity,SummonerMana,Restores 50% of your champion's maximum Mana. ...,False,False,False,False
8,14,10,Ignite,SummonerDot,"Ignites target enemy champion, dealing 70-410 ...",False,False,False,False
9,21,4,Barrier,SummonerBarrier,Shields your champion from 115-455 damage (dep...,False,False,False,False


### Komplett leere oder fast leere Spalten

In [18]:
leere_spalten = df_spells.columns[df_spells.isnull().all()]
fast_leere_spalten = df_spells.columns[df_spells.isnull().mean() > 0.9]

print("Komplett leere Spalten:", list(leere_spalten))
print("Fast leere Spalten (>90% NaN):", list(fast_leere_spalten))

Komplett leere Spalten: []
Fast leere Spalten (>90% NaN): []


### Datums-/Zeitspalten prüfen

In [19]:
datum_spalten = df_spells.select_dtypes(include=['datetime64', 'datetime64[ns]']).columns

if len(datum_spalten) == 0:
    print("Keine Datums-/Zeitspalten vorhanden.")
else:
    print("Datums-/Zeitspalten:", list(datum_spalten))
    for sp in datum_spalten:
        print(sp, "→ Min:", df_spells[sp].min(), "| Max:", df_spells[sp].max())

Keine Datums-/Zeitspalten vorhanden.


### Zeichenketten in kategorischen Spalten

In [20]:
kategorie_spalten = df_spells.select_dtypes(include=['object']).columns
print("Gefundene Textspalten:", list(kategorie_spalten))

Gefundene Textspalten: ['name', 'key', 'description']


In [21]:
kategorie_spalten = df_spells.select_dtypes(include=['object']).columns

if len(kategorie_spalten) == 0:
    print("Keine kategorischen (Text-)Spalten gefunden.")
else:
    for spalte in kategorie_spalten:
        print(f"Spalte: {spalte}")
        print(df_spells[spalte].str.len().describe())
        print("-" * 40)

Spalte: name
count    17.000000
mean     11.411765
std       9.240257
min       4.000000
25%       5.000000
50%       7.000000
75%      12.000000
max      30.000000
Name: name, dtype: float64
----------------------------------------
Spalte: key
count    17.000000
mean     17.058824
std       5.760540
min      11.000000
25%      13.000000
50%      15.000000
75%      18.000000
max      28.000000
Name: key, dtype: float64
----------------------------------------
Spalte: description
count     17.000000
mean     129.823529
std       61.991164
min       39.000000
25%       86.000000
50%      116.000000
75%      190.000000
max      199.000000
Name: description, dtype: float64
----------------------------------------


### Version

In [22]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\summoner_spell_info.json"
with open(pfad, encoding='utf-8') as f:
    roh = json.load(f)
spell_dict = roh['data']
spell_liste = list(spell_dict.values())
df_spells = pd.json_normalize(spell_liste)
df_spells['version'] = roh['version']

print("Verschiedene Patch-Versionen:")
print(df_spells['version'].value_counts())

Verschiedene Patch-Versionen:
version
7.17.2    17
Name: count, dtype: int64


### Summoner-Level-Bereich

In [23]:
df_spells['summonerLevel'].value_counts().sort_index()

summonerLevel
1     10
4      2
6      2
8      1
10     2
Name: count, dtype: int64

### Beschreibung auf Vollständigkeit

In [24]:
df_spells[df_spells['description'].isna() | (df_spells['description'].str.strip() == '')]

,id,summonerLevel,name,key,description,version


### Sonderzeichen in Namen

In [25]:
sonderzeichen = df_spells['name'].apply(lambda x: re.findall(r'[^a-zA-Z0-9\s]', x))
df_spells[sonderzeichen.apply(lambda x: len(x) > 0)]

,id,summonerLevel,name,key,description,version
10,30,1,To the King!,SummonerPoroRecall,Quickly travel to the Poro King's side.,7.17.2
13,33,1,Nexus Siege: Siege Weapon Slot,SummonerSiegeChampSelect1,"In Nexus Siege, Summoner Spells are replaced w...",7.17.2
14,34,1,Nexus Siege: Siege Weapon Slot,SummonerSiegeChampSelect2,"In Nexus Siege, Summoner Spells are replaced w...",7.17.2


### Key-Format

In [26]:
df_spells['key'].value_counts().sort_index()

key
SummonerBarrier                 1
SummonerBoost                   1
SummonerDarkStarChampSelect1    1
SummonerDarkStarChampSelect2    1
SummonerDot                     1
SummonerExhaust                 1
SummonerFlash                   1
SummonerHaste                   1
SummonerHeal                    1
SummonerMana                    1
SummonerPoroRecall              1
SummonerPoroThrow               1
SummonerSiegeChampSelect1       1
SummonerSiegeChampSelect2       1
SummonerSmite                   1
SummonerSnowball                1
SummonerTeleport                1
Name: count, dtype: int64

### Doppelte Beschreibungen

In [27]:
df_spells['description'].value_counts()[df_spells['description'].value_counts() > 1]

description
In Nexus Siege, Summoner Spells are replaced with Siege Weapon Slots. Spend Crystal Shards to buy single-use Siege Weapons from the item shop, then use your Summoner Spell keys to activate them!    2
Summoner spells are disabled in this mode.                                                                                                                                                            2
Name: count, dtype: int64

### Ausreißer Checks

In [28]:
# Die numerischen Spalten (id, key, summonerLevel) sind feste Werte pro Zauber. 
# Statistische Ausreißerprüfungen ergeben nur Fehlalarme, daher keine Ausreißer vorhanden

### Dataset 4: summoner_spell_info – Spezifische Checks
- **Sonderzeichen im Namen prüfen**
- **Key-Format-Prüfung (id, key, name)**
- **Leere Strings prüfen**
- **Doppelte id oder key prüfen**
- **Gleicher name für mehrere ids prüfen**
- **Negativwerte in id prüfen (z. B. -1)**
- **Test auf ungewöhnliche Inhalte prüfen**
- **Summoner Spells nach Modus oder Event-Typen kategorisieren (z. B. ARAM, Nexus Siege, Poro King)**

### Sonderzeichen im Namen

In [29]:
pattern = r"[^a-zA-Z0-9äöüÄÖÜß\s:!]"
sonderzeichen_df = df_spells[df_spells['name'].str.contains(pattern, regex=True)]

print("Summoner Spells mit Sonderzeichen im Namen:")
print(sonderzeichen_df[['id', 'name']])

Summoner Spells mit Sonderzeichen im Namen:
Empty DataFrame
Columns: [id, name]
Index: []


### Key-Format

In [30]:
id_check = df_spells['id'].apply(lambda x: isinstance(x, int))
print("Ungültige IDs (nicht vom Typ int):")
print(df_spells[~id_check][['id', 'key', 'name']])

key_check = df_spells['key'].str.startswith('Summoner')
print("\nKeys, die nicht mit 'Summoner' beginnen:")
print(df_spells[~key_check][['id', 'key', 'name']])

name_check = df_spells['name'].notna() & (df_spells['name'].str.strip() != "")
print("\nLeere oder fehlende Namen:")
print(df_spells[~name_check][['id', 'key', 'name']])

Ungültige IDs (nicht vom Typ int):
Empty DataFrame
Columns: [id, key, name]
Index: []

Keys, die nicht mit 'Summoner' beginnen:
Empty DataFrame
Columns: [id, key, name]
Index: []

Leere oder fehlende Namen:
Empty DataFrame
Columns: [id, key, name]
Index: []


### Leere Strings

In [31]:
leere_name = df_spells['name'].str.strip() == ""
leere_key = df_spells['key'].str.strip() == ""
leere_description = df_spells['description'].str.strip() == ""

print("Leere 'name'-Einträge:")
print(df_spells[leere_name][['id', 'name']])

print("\nLeere 'key'-Einträge:")
print(df_spells[leere_key][['id', 'key']])

print("\nLeere 'description'-Einträge:")
print(df_spells[leere_description][['id', 'description']])

Leere 'name'-Einträge:
Empty DataFrame
Columns: [id, name]
Index: []

Leere 'key'-Einträge:
Empty DataFrame
Columns: [id, key]
Index: []

Leere 'description'-Einträge:
Empty DataFrame
Columns: [id, description]
Index: []


### Doppelte id oder key

In [32]:
print("Doppelte 'id'-Werte:")
print(df_spells[df_spells.duplicated(subset='id', keep=False)][['id', 'key', 'name']])

print("\nDoppelte 'key'-Werte:")
print(df_spells[df_spells.duplicated(subset='key', keep=False)][['id', 'key', 'name']])

Doppelte 'id'-Werte:
Empty DataFrame
Columns: [id, key, name]
Index: []

Doppelte 'key'-Werte:
Empty DataFrame
Columns: [id, key, name]
Index: []


### Gleicher name für mehrere ids

In [33]:
mehrfach_ids = df_spells.groupby('name')['id'].nunique()
doppelte_namen = mehrfach_ids[mehrfach_ids > 1]

print("Namen mit mehreren unterschiedlichen IDs:")
print(doppelte_namen)

# Die doppelten Namen sind gewollt – sie stammen aus Sondermodi und verwenden unterschiedliche Slots

Namen mit mehreren unterschiedlichen IDs:
name
Disabled Summoner Spells          2
Nexus Siege: Siege Weapon Slot    2
Name: id, dtype: int64


### Negativwerte in id

In [34]:
negative_ids = df_spells[df_spells['id'] < 0]
print("Summoner Spells mit negativer ID:")
print(negative_ids[['id', 'key', 'name']])

Summoner Spells mit negativer ID:
Empty DataFrame
Columns: [id, key, name]
Index: []


### Text auf ungewöhnliche Inhalte

In [35]:
suchwörter = ['disabled', 'siege', 'poro', 'crystal shards', 'nexus', 'dark star']
unusual_df = df_spells[df_spells['description'].str.lower().str.contains('|'.join(suchwörter))]

print("Spells mit ungewöhnlichen Begriffen in der Beschreibung:")
print(unusual_df[['id', 'name', 'description']])

# So wie oben schon beschrieben gehören zu einem anderen Spielmodus somit ok.

Spells mit ungewöhnlichen Begriffen in der Beschreibung:
    id                            name  \
10  30                    To the King!   
11  31                       Poro Toss   
13  33  Nexus Siege: Siege Weapon Slot   
14  34  Nexus Siege: Siege Weapon Slot   
15  35        Disabled Summoner Spells   
16  36        Disabled Summoner Spells   

                                          description  
10            Quickly travel to the Poro King's side.  
11  Toss a Poro at your enemies. If it hits, you c...  
13  In Nexus Siege, Summoner Spells are replaced w...  
14  In Nexus Siege, Summoner Spells are replaced w...  
15         Summoner spells are disabled in this mode.  
16         Summoner spells are disabled in this mode.  


### Summoner Spells nach Modus oder Event-Typen kategorisieren

In [36]:
spezialwörter = ['siege', 'poro', 'nexus', 'dark star', 'king', 'crystal shards']
df_spells['event_modus'] = df_spells['description'].str.lower().apply(
    lambda x: 'Spezialmodus' if any(wort in x for wort in spezialwörter) else 'Standard'
)

print("Spells, die nur in Spezialmodi vorkommen:")
print(df_spells[df_spells['event_modus'] == 'Spezialmodus'][['id', 'name', 'description']])

# So wie oben schon beschrieben gehören zu einem anderen Spielmodus somit ok.

Spells, die nur in Spezialmodi vorkommen:
    id                            name  \
10  30                    To the King!   
11  31                       Poro Toss   
13  33  Nexus Siege: Siege Weapon Slot   
14  34  Nexus Siege: Siege Weapon Slot   

                                          description  
10            Quickly travel to the Poro King's side.  
11  Toss a Poro at your enemies. If it hits, you c...  
13  In Nexus Siege, Summoner Spells are replaced w...  
14  In Nexus Siege, Summoner Spells are replaced w...  


In [37]:
df_spells[df_spells['event_modus'] == 'Standard']

,id,summonerLevel,name,key,description,version,event_modus
0,1,6,Cleanse,SummonerBoost,Removes all disables (excluding suppression an...,7.17.2,Standard
1,3,4,Exhaust,SummonerExhaust,"Exhausts target enemy champion, reducing their...",7.17.2,Standard
2,4,8,Flash,SummonerFlash,Teleports your champion a short distance towar...,7.17.2,Standard
3,6,1,Ghost,SummonerHaste,Your champion gains increased Movement Speed a...,7.17.2,Standard
4,7,1,Heal,SummonerHeal,Restores 90-345 Health (depending on champion ...,7.17.2,Standard
5,11,10,Smite,SummonerSmite,Deals 390-1000 true damage (depending on champ...,7.17.2,Standard
6,12,6,Teleport,SummonerTeleport,"After channeling for 4.5 seconds, teleports yo...",7.17.2,Standard
7,13,1,Clarity,SummonerMana,Restores 50% of your champion's maximum Mana. ...,7.17.2,Standard
8,14,10,Ignite,SummonerDot,"Ignites target enemy champion, dealing 70-410 ...",7.17.2,Standard
9,21,4,Barrier,SummonerBarrier,Shields your champion from 115-455 damage (dep...,7.17.2,Standard


In [38]:
event_spells = df_spells[df_spells['event_modus'] == 'Spezialmodus']

print("Nur Spells aus Spezialmodi:")
print(event_spells[['id', 'name', 'description']])

Nur Spells aus Spezialmodi:
    id                            name  \
10  30                    To the King!   
11  31                       Poro Toss   
13  33  Nexus Siege: Siege Weapon Slot   
14  34  Nexus Siege: Siege Weapon Slot   

                                          description  
10            Quickly travel to the Poro King's side.  
11  Toss a Poro at your enemies. If it hits, you c...  
13  In Nexus Siege, Summoner Spells are replaced w...  
14  In Nexus Siege, Summoner Spells are replaced w...  


### Cross-Dataset-Checks
- **Kommen alle summoner_spell_ids aus dem Games-Datensatz auch in diesem Datensatz vor?**
- **Sind die Spells aus summoner_spell_info tatsächlich im Spiel nutzbar?**
- **Prüfung auf Platzhalter-Spells oder ungültige IDs (z. B. -1) im Games-Datensatz**
- **Summoner Spells nach Häufigkeit im Games-Datensatz zählen und mit Level-Voraussetzung vergleichen**
- **Sind die key-Werte (SummonerFlash, SummonerTeleport etc.) im Info-Datensatz 1:1 auflösbar aus dem Games-Datensatz?**

### Kommen alle summoner_spell_ids aus dem Games-Datensatz auch in diesem Datensatz vor?

In [39]:
df_games2 = pd.read_csv("games.csv")

spell_spalten = [
    't1_champ1_sum1', 't1_champ1_sum2',
    't1_champ2_sum1', 't1_champ2_sum2',
    't1_champ3_sum1', 't1_champ3_sum2',
    't1_champ4_sum1', 't1_champ4_sum2',
    't1_champ5_sum1', 't1_champ5_sum2',
    't2_champ1_sum1', 't2_champ1_sum2',
    't2_champ2_sum1', 't2_champ2_sum2',
    't2_champ3_sum1', 't2_champ3_sum2',
    't2_champ4_sum1', 't2_champ4_sum2',
    't2_champ5_sum1', 't2_champ5_sum2'
]
verwendete_ids = pd.unique(pd.concat([df_games2[spalte] for spalte in spell_spalten]))
spell_ids_info = df_spells['id'].unique()
nicht_verwendet = set(spell_ids_info) - set(verwendete_ids)

print("Spell-IDs aus dem Info-Datensatz, die NICHT im Games-Datensatz vorkommen:")
print(sorted(nicht_verwendet))

# Nicht verwendete Spell-IDs gehören zu Events oder alten Modi und sind korrekt im Datensatz enthalten.

Spell-IDs aus dem Info-Datensatz, die NICHT im Games-Datensatz vorkommen:
[np.int64(13), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36)]


### Sind die Spells aus summoner_spell_info tatsächlich im Spiel nutzbar?

In [40]:
unbenutzte_spells = df_spells[df_spells['id'].isin(nicht_verwendet)]

print("Spells, die im Games-Datensatz nicht vorkommen (also nicht nutzbar):")
print(unbenutzte_spells[['id', 'name', 'description']])

# Diese Spells gehören zu Sondermodi oder sind veraltet. Sie sind im Datensatz korrekt enthalten, werden aber in Ranked Games nicht verwendet.

Spells, die im Games-Datensatz nicht vorkommen (also nicht nutzbar):
    id                            name  \
7   13                         Clarity   
10  30                    To the King!   
11  31                       Poro Toss   
12  32                            Mark   
13  33  Nexus Siege: Siege Weapon Slot   
14  34  Nexus Siege: Siege Weapon Slot   
15  35        Disabled Summoner Spells   
16  36        Disabled Summoner Spells   

                                          description  
7   Restores 50% of your champion's maximum Mana. ...  
10            Quickly travel to the Poro King's side.  
11  Toss a Poro at your enemies. If it hits, you c...  
12  Throw a snowball in a straight line at your en...  
13  In Nexus Siege, Summoner Spells are replaced w...  
14  In Nexus Siege, Summoner Spells are replaced w...  
15         Summoner spells are disabled in this mode.  
16         Summoner spells are disabled in this mode.  


### Prüfung auf Platzhalter-Spells oder ungültige IDs im Games-Datensatz

In [41]:
gültige_ids = set(df_spells['id'].unique())

spell_spalten = [
    't1_champ1_sum1', 't1_champ1_sum2', 't1_champ2_sum1', 't1_champ2_sum2',
    't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4_sum1', 't1_champ4_sum2',
    't1_champ5_sum1', 't1_champ5_sum2', 't2_champ1_sum1', 't2_champ1_sum2',
    't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3_sum1', 't2_champ3_sum2',
    't2_champ4_sum1', 't2_champ4_sum2', 't2_champ5_sum1', 't2_champ5_sum2'
]
alle_games_ids = pd.unique(pd.concat([df_games2[spalte] for spalte in spell_spalten]))
ungültige_ids = set(alle_games_ids) - gültige_ids

print("Ungültige oder unbekannte Spell-IDs im Games-Datensatz:")
print(sorted(ungültige_ids))

Ungültige oder unbekannte Spell-IDs im Games-Datensatz:
[]


### Summoner Spells nach Häufigkeit im Games-Datensatz zählen und mit Level-Voraussetzung vergleichen

In [42]:
spell_spalten = [
    't1_champ1_sum1', 't1_champ1_sum2', 't1_champ2_sum1', 't1_champ2_sum2',
    't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4_sum1', 't1_champ4_sum2',
    't1_champ5_sum1', 't1_champ5_sum2', 't2_champ1_sum1', 't2_champ1_sum2',
    't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3_sum1', 't2_champ3_sum2',
    't2_champ4_sum1', 't2_champ4_sum2', 't2_champ5_sum1', 't2_champ5_sum2'
]
alle_spells = pd.concat([df_games2[spalte] for spalte in spell_spalten])
spell_counts = alle_spells.value_counts().reset_index()
spell_counts.columns = ['id', 'verwendung']
verwendung_mit_level = spell_counts.merge(df_spells[['id', 'name', 'summonerLevel']], on='id', how='left')

print("Spell-Häufigkeit und zugehörige Mindestlevel:")
print(verwendung_mit_level.sort_values(by='verwendung', ascending=False))

# Hier ist das Accountlevel gemeint.

Spell-Häufigkeit und zugehörige Mindestlevel:
   id  verwendung      name  summonerLevel
0   4      504121     Flash              8
1  12      109401  Teleport              6
2   7      108043      Heal              1
3  11      103062     Smite             10
4  14       88258    Ignite             10
5   3       84576   Exhaust              4
6  21       14880   Barrier              4
7   6       14305     Ghost              1
8   1        3154   Cleanse              6


### Sind die key-Werte (SummonerFlash, SummonerTeleport etc.) im Info-Datensatz 1:1 auflösbar aus dem Games-Datensatz?

In [43]:
spell_spalten = [
    't1_champ1_sum1', 't1_champ1_sum2', 't1_champ2_sum1', 't1_champ2_sum2',
    't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4_sum1', 't1_champ4_sum2',
    't1_champ5_sum1', 't1_champ5_sum2', 't2_champ1_sum1', 't2_champ1_sum2',
    't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3_sum1', 't2_champ3_sum2',
    't2_champ4_sum1', 't2_champ4_sum2', 't2_champ5_sum1', 't2_champ5_sum2'
]
games_ids = pd.unique(pd.concat([df_games2[spalte] for spalte in spell_spalten]))
keys_mapping = df_spells[df_spells['id'].isin(games_ids)][['id', 'key']]
fehlende_keys = set(games_ids) - set(keys_mapping['id'])

print("Spell-IDs aus dem Games-Datensatz ohne zugeordneten key im Info-Datensatz:")
print(sorted(fehlende_keys))

Spell-IDs aus dem Games-Datensatz ohne zugeordneten key im Info-Datensatz:
[]


### Abspeichern und eine Copi erstellen

In [44]:
df_spells.to_csv("Summoner_spell2.csv", index=False)
print("Sicherheitskopie erfolgreich gespeichert: 'Summoner_spell2.csv'")

Sicherheitskopie erfolgreich gespeichert: 'Summoner_spell2.csv'
